In [1]:
library(keras)
library(repr)

In [2]:
library(keras)
library(repr)

In [3]:
data.bkg  <- read.csv("data/P_LHCB_LambdaB/background.csv")
data.sig  <- read.csv("data/P_LHCB_LambdaB/MC_signal.csv")
data.lhcb <- read.csv("data/P_LHCB_LambdaB/data_lhcb.csv")

In [4]:
data.cutted.bkg  <-  data.bkg  %>% dplyr::filter( (Lambda_b0_MM_F > 5550 &  Lambda_b0_MM_F < 5680 ) |  DeltaM_F < 360)
data.cutted.sig  <-  data.sig  %>% dplyr::filter( (Lambda_b0_MM_F > 5550 &  Lambda_b0_MM_F < 5680 ) |  DeltaM_F < 360)
data.cutted.lhcb <-  data.lhcb %>% dplyr::filter( (Lambda_b0_MM_F > 5550 &  Lambda_b0_MM_F < 5680 ) |  DeltaM_F < 360)

In [29]:
# test set <- 400 sig 400 bkg

sel_features <- c(3, 6, 7, 8, 9, 10, 11, 12, 14, 17, 18)
nt_bkg <- 400; nt_sig <- 400

x_test_bkg  <- as.matrix(data.bkg[1:nt_bkg, sel_features])
x_test_sig  <- as.matrix(data.sig[1:nt_sig, sel_features])

y_test_bkg  <- as.vector(matrix(0, nrow=nt_bkg, ncol=1))
y_test_sig  <- as.vector(matrix(1, nrow=nt_sig, ncol=1))

x_test  <- rbind(x_test_bkg  , x_test_sig  )
y_test  <-     c(y_test_bkg  , y_test_sig  )

x_test  <- scale(x_test )

shuffle_test  <- sample(nrow(x_test ))
x_test  <- x_test [shuffle_test,]
y_test  <- y_test [shuffle_test] 



In [30]:
#training set, dimension give with function

x_train_bkg_full <- as.matrix(data.bkg[-1:-nt_bkg, sel_features])
x_train_sig_full <- as.matrix(data.sig[-1:-nt_sig, sel_features])

y_train_bkg_full <- as.vector(matrix(0, nrow=nrow(x_train_bkg_full), ncol=1))
y_train_sig_full <- as.vector(matrix(1, nrow=nrow(x_train_sig_full), ncol=1))

get_train_data <- function(n_bkg, n_sig=1090 ){
    x_bkg <- x_train_bkg_full[1:n_bkg, ]
    x_sig <- x_train_sig_full[1:n_sig, ]
    x_t   <- rbind(x_bkg, x_sig)
    y_t   <- c(y_train_bkg_full[1:n_bkg], y_train_sig_full[1:n_sig])
    sffl  <- sample(nrow(x_t))
    
    x_t   <- x_t[sffl,]
    x_t   <- scale(x_t)
    y_t   <- y_t[sffl] 
    return (list(x_train=x_t, y_train=y_t))
}


In [31]:
# lhcb data
x_lhcb     <- as.matrix(       data.lhcb[, sel_features])
x_cut_lhcb <- as.matrix(data.cutted.lhcb[, sel_features])

x_lhcb     <- scale(x_lhcb)
x_cut_lhcb <- scale(x_cut_lhcb)


In [104]:
build_model <- function(params){
    model <- keras_model_sequential()
    model %>%
      layer_dense(units = params$unit1, activation = 'relu', 
                  input_shape = c(params$nFeat), 
                  kernel_regularizer=regularizer_l1_l2(l1=params$l1_coeff, l2=params$l2_coeff)) %>%
      layer_dropout(rate = params$dropout1) %>%
      layer_dense(units = params$unit2, activation = 'relu',  
                  kernel_regularizer=regularizer_l1_l2(l1=params$l1_coeff, l2=params$l2_coeff)) %>%
      layer_dropout(rate = params$dropout2) %>%
      layer_dense(units = params$unit3, activation = 'relu', 
                  kernel_regularizer=regularizer_l1_l2(l1=params$l1_coeff, l2=params$l2_coeff)) %>%
      layer_dropout(rate = params$dropout3) %>%
      layer_dense(units = 1, activation = 'sigmoid')

    model %>% compile(
      loss = 'binary_crossentropy',
      optimizer = optimizer_rmsprop(),
      metrics = c('accuracy')
    )
    return(model)
}

ModelParams <- flags(
    flag_numeric("dropout1", 0.1),
    flag_numeric("dropout2", 0.1),
    flag_numeric("dropout3", 0.1),
    flag_numeric("unit1", 128),
    flag_numeric("unit2", 64),
    flag_numeric("unit3", 32),
    flag_numeric("nFeat", length(sel_features)),
    flag_numeric("l1_coeff", 1e-4),
    flag_numeric("l2_coeff", 1e-5)
)

In [ ]:
model <- build_model(ModelParams)
data  <- get_train_data(1000) 
history  <- model %>% fit (data$x_train, data$y_train, verbose=2,
                     epochs=10, batch_size=10, validation_split=0.2)
model %>% evaluate(x_test, y_test)
plot(history)

In [109]:
save_model_hdf5(model, "Models/Test/TestModel.hdf5")

In [83]:
y_cut_lhcb_pred <- model %>% predict(x_cut_lhcb)

In [ ]:
hist(data.cutted.lhcb$Lambda_b0_MM_F[y_cut_lhcb_pred>0.0],   breaks=100, col="firebrick3", xlab="Lb0 Mass", main="LHCb data",  probability=FALSE)
hist(data.cutted.lhcb$Lambda_b0_MM_F[y_cut_lhcb_pred>0.99], breaks=100, add=TRUE, col='blue')
#hist(data.cutted.lhcb$Lambda_b0_MM_F[y_cut_lhcb_pred>0.2], breaks=100, add=TRUE, col='green')
#hist(data.cutted.lhcb$Lambda_b0_MM_F[y_cut_lhcb_pred>0.3], breaks=100, add=TRUE, col='orange')
#hist(data.cutted.lhcb$Lambda_b0_MM_F[y_cut_lhcb_pred>0.5], breaks=100, add=TRUE, col='black')


In [160]:
cv_tune_builder <- function (builder, params, x, y, cv=4, name="Test"){
    l=as.integer(nrow(x)/cv)
    accs <- NULL
    for(i in 0:(cv-1)){
        x_tr <- x[-(i*l+1):-(l*(i+1)),]; x_cv <- x[(i*l+1):(l*(i+1)),]; 
        y_tr <- y[-(i*l+1):-(l*(i+1)) ]; y_cv <- y[(i*l+1):(l*(i+1)) ];
        model <- builder(params)
        eph    = ifelse('epochs'     %in% names(params), params$epochs     , 30  )
        b_size = ifelse('batch_size' %in% names(params), params$batch_size , 10 )
        hist  <- model %>% fit (x_tr, y_tr,  epochs = eph, batch_size = b_size, validation_data = list(x_cv, y_cv))
        stats <- model %>% evaluate(x_test, y_test)
        accs <- c(accs, stats[[2]])
        mName <- paste(name, "_cv",i,".hdf5", sep="")
        save_model_hdf5(model, mName)
    }
    return (accs)
}


tune_builder <- function(builder, par_default, par_list, cv=4, verbose=FALSE, path="Models/Test/"){
    if(!dir.exists(path)){
        dir.create(path)
    }
    grid = expand.grid(par_list)
    mean_accs <- NULL
    best_acc <- 0.0
    par_best <- rlang::duplicate(par_default, shallow=FALSE)
    for(i in 1:nrow(grid)){
        par_copy <- rlang::duplicate(par_default, shallow=FALSE)
        mName <- "Model"
        for (name in names(par_list)){
            par_copy[[name]] <- grid[[name]][i]
            mName <- paste(mName,"_",name, grid[[name]][i], sep="")
        }
        data <- get_train_data(par_copy$n_bkg, par_copy$n_sig) #get dataset
        x <- data$x_train; y<-data$y_train;
        acc <- cv_tune_builder(builder, par_copy, x, y, cv, paste(path,mName, sep=""))
        mean_accs <- c(mean_accs, mean(acc))
        if(verbose){
            print("Parameters: ")
            print(par_copy)
            print("Accuracies:")
            print(acc)
            print(paste("Mean: ", mean(acc)))
        }
        if (mean(acc)>best_acc){
            best_acc <- mean(acc)
            par_best <- rlang::duplicate(par_copy, shallow=FALSE)
        }        
    }
    if(verbose){
        print("Best Parameters: ")
        print(par_best)
        print(paste("Accuracy:", best_acc))
    }
    grid$Mean_acc <- mean_accs
    #return(par_best)
    return (grid)
}

In [153]:
TunerParams <- flags(
    flag_numeric("dropout1"   , 0.1                 ),
    flag_numeric("dropout2"   , 0.1                 ),
    flag_numeric("dropout3"   , 0.1                 ),
    flag_numeric("unit1"      , 128                 ),
    flag_numeric("unit2"      , 64                  ),
    flag_numeric("unit3"      , 32                  ),
    flag_numeric("nFeat"      , length(sel_features)),
    flag_numeric("l1_coeff"   , 1e-4                ),
    flag_numeric("l2_coeff"   , 1e-5                ),
    flag_string ("epochs"     , 30                  ),  
    flag_string ("batch_size" , 10                  ),
    flag_numeric("n_bkg"      , 1000                ),
    flag_numeric("n_sig"      , 1090                )            
)

In [148]:
NBkgList <- list(n_bkg= c(500 , 750 , 1000, 1500,
                          2000, 2500, 3000), 
                 epochs=(20))

tuned_grid <- tune_builder(build_model, TunerParams, NBkgList, 4, FALSE, "Models/BkgNum/" )

print(tuned_grid)

#

  n_bkg epochs  Mean_acc
1   500     20 0,7940625
2   750     20 0,8340625
3  1000     20 0,8443750
4  1500     20 0,8265625
5  2000     20 0,7906250
6  2500     20 0,7771875
7  3000     20 0,7415625


In [155]:
#Batch_Epoch_List <- list(batch_size=c(1, 5, 10,20,50, 100), 
#               epochs=c(5,10,20))
Batch_Epoch_List <- list(batch_size=c( 10), 
               epochs=c(5,10,20, 30, 50, 75, 100, 150, 200, 300))

tuned_grid <- tune_builder(build_model, TunerParams, Batch_Epoch_List, 4, FALSE, "Models/Batch_Epoch/" )

print(tuned_grid)

Model_batch_size10
Model_batch_size10_epochs5
Model_batch_size10
Model_batch_size10_epochs10
Model_batch_size10
Model_batch_size10_epochs20
Model_batch_size10
Model_batch_size10_epochs30
Model_batch_size10
Model_batch_size10_epochs50
Model_batch_size10
Model_batch_size10_epochs75
Model_batch_size10
Model_batch_size10_epochs100
Model_batch_size10
Model_batch_size10_epochs150
Model_batch_size10
Model_batch_size10_epochs200
Model_batch_size10
Model_batch_size10_epochs300


   batch_size epochs  Mean_acc
1          10      5 0,8456250
2          10     10 0,8450000
3          10     20 0,8409375
4          10     30 0,8406250
5          10     50 0,8350000
6          10     75 0,8328125
7          10    100 0,8312500
8          10    150 0,8300000
9          10    200 0,8253125
10         10    300 0,8231250


In [158]:
getOptimizer <- function (params){
    if (params$optimizer == "rmsprop"){
        return (optimizer_rmsprop(lr = params$lr, rho=params$rho))}
    if (params$optimizer == "adam"){
        return (optimizer_adam(lr = params$lr, beta_1 = params$beta_1, 
                                               beta_2 = params$beta_2 ))}
    if (params$optimizer == "nadam"){
        return (optimizer_nadam(lr = params$lr, beta_1 = params$beta_1, 
                                                beta_2 = params$beta_2 ))}
    if (params$optimizer == "sgd"){
        return (optimizer_sgd(lr = params$lr, momentum = params$momentum, 
                               decay = 1e-6, nesterov = TRUE)) }
}


Archi_build_model <- function(params){
    model <- keras_model_sequential()
    model %>%
      layer_dense(units = params$unit1, activation = params$act_dense, 
                  input_shape = c(params$nFeat), 
                  kernel_regularizer=regularizer_l1_l2(l1=params$l1_coeff, l2=params$l2_coeff)) %>%
      layer_dropout(rate = params$dropout1) %>%
      layer_dense(units = params$unit2, activation = params$act_dense,  
                  kernel_regularizer=regularizer_l1_l2(l1=params$l1_coeff, l2=params$l2_coeff)) %>%
      layer_dropout(rate = params$dropout2) %>%
      layer_dense(units = params$unit3, activation = params$act_dense, 
                  kernel_regularizer=regularizer_l1_l2(l1=params$l1_coeff, l2=params$l2_coeff)) %>%
      layer_dropout(rate = params$dropout3) %>%
      layer_dense(units = 1, activation = params$act_final)

    model %>% compile(
      loss = params$loss,
      optimizer = getOptimizer(params),
      metrics = c('accuracy')
    )
    return(model)
}

ArchiParams <- flags(
    flag_numeric("dropout1"   , 0.1                   ),
    flag_numeric("dropout2"   , 0.1                   ),
    flag_numeric("dropout3"   , 0.1                   ),
    flag_numeric("unit1"      , 128                   ),
    flag_numeric("unit2"      , 64                    ),
    flag_numeric("unit3"      , 32                    ),
    flag_numeric("nFeat"      , length(sel_features)  ),
    flag_numeric("l1_coeff"   , 1e-4                  ),
    flag_numeric("l2_coeff"   , 1e-5                  ),
    flag_string ("epochs"     , 10                    ),  
    flag_string ("batch_size" , 5                     ),
    flag_numeric("n_bkg"      , 1000                  ),
    flag_numeric("n_sig"      , 1090                  ),
    flag_string ("act_dense"  , "relu"                ),
    flag_string ("act_final"  , "sigmoid"             ),
    flag_string ("loss"       , "binary_crossentropy" ),
    flag_string ("optimizer"  , "rmsprop"             ),
    flag_numeric("lr"         , 0.001                 ), #learing rate for the optimizers
    flag_numeric("rho"        , 0.9                   ), #rms rho
    flag_numeric("beta_1"     , 0.9                   ), #beta1 adam/nadam
    flag_numeric("beta_2"     , 0.999                 ), #beta2 adam/nadam
    flag_numeric("momentum"   , 0.9                   ) #momentum sgd
)

In [161]:
Units_List <- list(unit1=c(256,128, 64),
                   unit2=c(128, 64, 32),
                   unit3=c( 64, 32, 16))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams, Units_List, 4, FALSE,
                           "Models/Units/" )

print(tuned_grid)

   unit1 unit2 unit3  Mean_acc
1    256   128    64 0,8400000
2    128   128    64 0,8484375
3     64   128    64 0,8471875
4    256    64    64 0,8437500
5    128    64    64 0,8434375
6     64    64    64 0,8400000
7    256    32    64 0,8440625
8    128    32    64 0,8425000
9     64    32    64 0,8475000
10   256   128    32 0,8453125
11   128   128    32 0,8478125
12    64   128    32 0,8503125
13   256    64    32 0,8353125
14   128    64    32 0,8462500
15    64    64    32 0,8434375
16   256    32    32 0,8384375
17   128    32    32 0,8406250
18    64    32    32 0,8456250
19   256   128    16 0,8440625
20   128   128    16 0,8462500
21    64   128    16 0,8368750
22   256    64    16 0,8478125
23   128    64    16 0,8462500
24    64    64    16 0,8440625
25   256    32    16 0,8468750
26   128    32    16 0,8406250
27    64    32    16 0,8478125


In [162]:
Dropout_List <- list(dropout1=c(0.1, 0.2, 0.3),
                     dropout2=c(0.1, 0.2, 0.3),
                     dropout3=c(0.1, 0.2, 0.3))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams, Dropout_List, 4, FALSE,
                           "Models/Dropouts/" )

print(tuned_grid)

   dropout1 dropout2 dropout3  Mean_acc
1       0,1      0,1      0,1 0,8437500
2       0,2      0,1      0,1 0,8462500
3       0,3      0,1      0,1 0,8437500
4       0,1      0,2      0,1 0,8431250
5       0,2      0,2      0,1 0,8406250
6       0,3      0,2      0,1 0,8453125
7       0,1      0,3      0,1 0,8481250
8       0,2      0,3      0,1 0,8450000
9       0,3      0,3      0,1 0,8512500
10      0,1      0,1      0,2 0,8450000
11      0,2      0,1      0,2 0,8453125
12      0,3      0,1      0,2 0,8384375
13      0,1      0,2      0,2 0,8381250
14      0,2      0,2      0,2 0,8437500
15      0,3      0,2      0,2 0,8440625
16      0,1      0,3      0,2 0,8478125
17      0,2      0,3      0,2 0,8446875
18      0,3      0,3      0,2 0,8459375
19      0,1      0,1      0,3 0,8406250
20      0,2      0,1      0,3 0,8503125
21      0,3      0,1      0,3 0,8409375
22      0,1      0,2      0,3 0,8350000
23      0,2      0,2      0,3 0,8468750
24      0,3      0,2      0,3 0,8381250


In [166]:
ArchiParams_TunedUnit <- flags(
    flag_numeric("dropout1"   , 0.3                   ),
    flag_numeric("dropout2"   , 0.3                   ),
    flag_numeric("dropout3"   , 0.1                   ),
    flag_numeric("unit1"      , 64                    ),
    flag_numeric("unit2"      , 128                   ),
    flag_numeric("unit3"      , 32                    ),
    flag_numeric("nFeat"      , length(sel_features)  ),
    flag_numeric("l1_coeff"   , 1e-4                  ),
    flag_numeric("l2_coeff"   , 1e-5                  ),
    flag_string ("epochs"     , 1                    ),  
    flag_string ("batch_size" , 100                    ),
    flag_numeric("n_bkg"      , 1000                  ),
    flag_numeric("n_sig"      , 1090                  ),
    flag_string ("act_dense"  , "relu"                ),
    flag_string ("act_final"  , "sigmoid"             ),
    flag_string ("loss"       , "binary_crossentropy" ),
    flag_string ("optimizer"  , "rmsprop"             ),
    flag_numeric("lr"         , 0.001                 ), #learing rate for the optimizers
    flag_numeric("rho"        , 0.9                   ), #rms rho
    flag_numeric("beta_1"     , 0.9                   ), #beta1 adam/nadam
    flag_numeric("beta_2"     , 0.999                 ), #beta2 adam/nadam
    flag_numeric("momentum"   , 0.9                   ) #momentum sgd
)

In [ ]:
Activation_List <- list(act_dense=c( "relu"   ,  "elu", "selu", "linear"),
                        act_final=c( "sigmoid", "softmax", "hard_sigmoid" ))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams_TunedUnit, Units_List, 2, FALSE,
                           "Models/Test/" )

print(tuned_grid)

In [ ]:
Loss_List <- list(loss=c( "binary_crossentropy", "poisson",
                              "kullback_leibler_divergence", "hinge",
                              "mean_squared_error", "mean_absolute_error"))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams_TunedUnit, Loss_List, 4, FALSE,
                           "Models/Loss/" )

print(tuned_grid)

In [ ]:
ArchiParams_TunedLayers <- flags(
    flag_numeric("dropout1"   , 0.3                   ),
    flag_numeric("dropout2"   , 0.3                   ),
    flag_numeric("dropout3"   , 0.1                   ),
    flag_numeric("unit1"      , 64                    ),
    flag_numeric("unit2"      , 128                   ),
    flag_numeric("unit3"      , 32                    ),
    flag_numeric("nFeat"      , length(sel_features)  ),
    flag_numeric("l1_coeff"   , 1e-4                  ),
    flag_numeric("l2_coeff"   , 1e-5                  ),
    flag_string ("epochs"     , 30                    ),  
    flag_string ("batch_size" , 10                    ),
    flag_numeric("n_bkg"      , 1000                  ),
    flag_numeric("n_sig"      , 1090                  ),
    flag_string ("act_dense"  , "relu"                ),
    flag_string ("act_final"  , "sigmoid"             ),
    flag_string ("loss"       , "binary_crossentropy" ),
    flag_string ("optimizer"  , "rmsprop"             ),
    flag_numeric("lr"         , 0.001                 ), #learing rate for the optimizers
    flag_numeric("rho"        , 0.9                   ), #rms rho
    flag_numeric("beta_1"     , 0.9                   ), #beta1 adam/nadam
    flag_numeric("beta_2"     , 0.999                 ), #beta2 adam/nadam
    flag_numeric("momentum"   , 0.9                   ) #momentum sgd
)

In [ ]:
RMS_List <-  list(optimizer="rmsprop",
                  lr = c(0.1,  0.01,  0.001,  1e-4),
                  rho= c(0.99, 0.95,    0.9,  0.85))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams_TunedLayers, RMS_List, 4, FALSE,
                           "Models/Optimizers/RMS/" )

print(tuned_grid)

In [ ]:
ADAM_List <-  list(optimizer="adam",
                  lr     = c(0.02,  0.002,  2e-4),
                  beta_1 = c(0.85,    0.9,  0.95),
                  beta_2 = c(0.99,  0.999       ))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams_TunedLayers, ADAM_List, 4, FALSE,
                           "Models/Optimizers/ADAM/" )

print(tuned_grid)

In [ ]:
NADAM_List <-  list(optimizer="nadam",
                  lr     = c(0.02,  0.002,  2e-4),
                  beta_1 = c(0.85,    0.9,  0.95),
                  beta_2 = c(0.99,  0.999       ))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams_TunedLayers, NADAM_List, 4, FALSE,
                           "Models/Optimizers/NADAM/" )

print(tuned_grid)

In [ ]:
SGD_List <-   list(optimizer="sgd",
                   lr       = c(0.1,  0.01,  0.001,  1e-4),
                   momentum = c(0.99, 0.95,    0.9,  0.85))

tuned_grid <- tune_builder(Archi_build_model, ArchiParams_TunedLayers, SGD_List, 4, FALSE,
                           "Models/Optimizers/SGD/" )

print(tuned_grid)

In [ ]:
ArchiParams_TunedOpt <- flags(
    flag_numeric("dropout1"   , 0.3                   ),
    flag_numeric("dropout2"   , 0.3                   ),
    flag_numeric("dropout3"   , 0.1                   ),
    flag_numeric("unit1"      , 64                    ),
    flag_numeric("unit2"      , 128                   ),
    flag_numeric("unit3"      , 32                    ),
    flag_numeric("nFeat"      , length(sel_features)  ),
    flag_numeric("l1_coeff"   , 1e-4                  ),
    flag_numeric("l2_coeff"   , 1e-5                  ),
    flag_string ("epochs"     , 30                    ),  
    flag_string ("batch_size" , 10                    ),
    flag_numeric("n_bkg"      , 1000                  ),
    flag_numeric("n_sig"      , 1090                  ),
    flag_string ("act_dense"  , "relu"                ),
    flag_string ("act_final"  , "sigmoid"             ),
    flag_string ("loss"       , "binary_crossentropy" ),
    flag_string ("optimizer"  , "rmsprop"             ),
    flag_numeric("lr"         , 0.001                 ), #learing rate for the optimizers
    flag_numeric("rho"        , 0.9                   ), #rms rho
    flag_numeric("beta_1"     , 0.9                   ), #beta1 adam/nadam
    flag_numeric("beta_2"     , 0.999                 ), #beta2 adam/nadam
    flag_numeric("momentum"   , 0.9                   ) #momentum sgd
)